In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import sys
sys.setrecursionlimit(10000)
%matplotlib qt5

In [3]:
import cellbgnet
import cellbgnet.utils
from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel

In [4]:
param_file = '../../cellbgnet/utils/reference_files/reference.yaml'
param = cellbgnet.utils.param_io.ParamHandling().load_params(param_file)

In [5]:
psf_params = param.PSF.to_dict()
simulation_params = param.Simulation.to_dict()
hardware_params = param.Hardware.to_dict()
eval_params = param.Evaluation.to_dict()
train_size = simulation_params['train_size']

In [6]:
model = cellbgnet.model.CellBGModel(param)

training sliding windows on camera chip:
Area num: 0, field_xy: [0, 127, 0, 127]
Area num: 1, field_xy: [114, 241, 0, 127]
Area num: 2, field_xy: [228, 355, 0, 127]
Area num: 3, field_xy: [342, 469, 0, 127]
Area num: 4, field_xy: [456, 583, 0, 127]
Area num: 5, field_xy: [570, 697, 0, 127]
Area num: 6, field_xy: [684, 811, 0, 127]
Area num: 7, field_xy: [798, 925, 0, 127]
Area num: 8, field_xy: [912, 1039, 0, 127]
Area num: 9, field_xy: [1026, 1153, 0, 127]
Area num: 10, field_xy: [1140, 1267, 0, 127]
Area num: 11, field_xy: [1174, 1301, 0, 127]
Area num: 12, field_xy: [0, 127, 114, 241]
Area num: 13, field_xy: [114, 241, 114, 241]
Area num: 14, field_xy: [228, 355, 114, 241]
Area num: 15, field_xy: [342, 469, 114, 241]
Area num: 16, field_xy: [456, 583, 114, 241]
Area num: 17, field_xy: [570, 697, 114, 241]
Area num: 18, field_xy: [684, 811, 114, 241]
Area num: 19, field_xy: [798, 925, 114, 241]
Area num: 20, field_xy: [912, 1039, 114, 241]
Area num: 21, field_xy: [1026, 1153, 114, 24

In [7]:
model.init_recorder()
model.init_eval_data()

Eval image generation: 100%|████████████████████| 30/30 [00:12<00:00,  2.44it/s]


eval images shape: (30, 1041, 1302) contain 21397 molecules,


In [8]:
model.evaluation_params['eval_imgs'].shape

(30, 1041, 1302)

In [9]:
simulation_params

{'train_size': 128,
 'train_type': 'cells',
 'psf_intensity_normalization': True,
 'density': 0.003125,
 'non_cell_density': 10,
 'margin_empty': 0.05,
 'z_prior': [-1, 1],
 'survival_prob': None,
 'min_photon': 0.25,
 'camera': 'sCMOS',
 'camera_chip_size': [1041, 1302],
 'perlin_noise': False,
 'perlin_noise_factor': 0.2,
 'perlin_noise_res': 64,
 'baseline': 103.0,
 'bg_values': 112.0,
 'e_per_adu': 0.4,
 'qe': 0.95,
 'spurious_c': 0.0,
 'sig_read': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_8865/RN_e_venus.tiff',
 'robust_training': False,
 'cell_masks_dir': '/mnt/sda1/SMLAT/data/real_data/replisome_dots/pool_3024/phase_venus_mask/',
 'cell_masks_filetype': '.png',
 'edt_noise_map_path': '/mnt/sda1/SMLAT/data/real_data/replisome_dots/pool_3024/edt_noise.pkl',
 'min_edt': 0,
 'max_edt': 6,
 'augment': True}

In [10]:
start_time = time.time()
model.fit(batch_size=32, max_iters=20000, print_output=True, print_freq=100)
duration = time.time() - start_time

Started training .... 
 || Cost: 14160.809 || Time Upd.: 439.2 ms  || BatchNr.: 100
 || Cost: 226.445 || Time Upd.: 431.2 ms  || BatchNr.: 200
 || Cost: 127.452 || Time Upd.: 419.2 ms  || BatchNr.: 300
 || Cost: 105.990 || Time Upd.: 428.3 ms  || BatchNr.: 400
 || Cost: 63.721 || Time Upd.: 455.5 ms  || BatchNr.: 500
 || Cost: 52.722 || Time Upd.: 427.3 ms  || BatchNr.: 600
 || Cost: 48.468 || Time Upd.: 539.5 ms  || BatchNr.: 700
 || Cost: 44.057 || Time Upd.: 406.7 ms  || BatchNr.: 800
 || Cost: 38.868 || Time Upd.: 416.8 ms  || BatchNr.: 900
 || Cost: 35.418 || Time Upd.: 417.9 ms  || BatchNr.: 1000
processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10631FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10631 ,preds: 6733
JoR: 1.158 || Eff_3d: 42.043 || Jaccard: 0.631 || Factor: 355.888 || RMSE_lat: 54.496 || RMSE_ax: 67.794 || RMSE_x: 33.1

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10631FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10631 ,preds: 7061
JoR: 1.380 || Eff_3d: 49.113 || Jaccard: 0.661 || Factor: 348.961 || RMSE_lat: 47.869 || RMSE_ax: 53.107 || RMSE_x: 34.495 || RMSE_y: 33.189 || Cost: 24.116 || Recall: 0.662 || Precision: 0.997 || BatchNr.: 2200 || Time Upd.: 445.0 ms 
 || Count Cost: 16.2064 || Localization Cost: 11.0769 || Bg Cost: 0.0539 || P_locs Cost: 21.9397saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 49.113 || 47.869 || 53.107 || 2200


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10631FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10631 ,preds: 7481
JoR: 1.439 || Eff_3d: 51.250

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10631FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10631 ,preds: 7983
JoR: 1.559 || Eff_3d: 54.893 || Jaccard: 0.745 || Factor: 343.380 || RMSE_lat: 47.771 || RMSE_ax: 50.894 || RMSE_x: 34.152 || RMSE_y: 33.403 || Cost: 21.712 || Recall: 0.747 || Precision: 0.995 || BatchNr.: 3400 || Time Upd.: 414.8 ms 
 || Count Cost: 15.8115 || Localization Cost: 9.6199 || Bg Cost: 0.0448 || P_locs Cost: 21.3881saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 54.893 || 47.771 || 50.894 || 3400


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10631FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10631 ,preds: 7898
JoR: 1.573 || Eff_3d: 54.914 

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10631FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10631 ,preds: 8344
JoR: 1.616 || Eff_3d: 56.236 || Jaccard: 0.779 || Factor: 369.747 || RMSE_lat: 48.208 || RMSE_ax: 52.977 || RMSE_x: 35.524 || RMSE_y: 32.589 || Cost: 18.594 || Recall: 0.782 || Precision: 0.996 || BatchNr.: 4600 || Time Upd.: 423.2 ms 
 || Count Cost: 16.1477 || Localization Cost: 6.8245 || Bg Cost: 0.0427 || P_locs Cost: 20.5102saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 56.236 || 48.208 || 52.977 || 4600


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10631FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10631 ,preds: 7985
JoR: 1.571 || Eff_3d: 54.552 

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10631FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10631 ,preds: 8356
JoR: 1.642 || Eff_3d: 56.825 || Jaccard: 0.777 || Factor: 373.445 || RMSE_lat: 47.336 || RMSE_ax: 51.529 || RMSE_x: 35.174 || RMSE_y: 31.678 || Cost: 17.893 || Recall: 0.781 || Precision: 0.994 || BatchNr.: 5800 || Time Upd.: 417.9 ms 
 || Count Cost: 15.2901 || Localization Cost: 6.7076 || Bg Cost: 0.0383 || P_locs Cost: 20.2626saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 56.825 || 47.336 || 51.529 || 5800


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10631FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10631 ,preds: 8168
JoR: 1.651 || Eff_3d: 56.901 

...

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10631FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10631 ,preds: 8594
JoR: 1.778 || Eff_3d: 60.086 || Jaccard: 0.804 || Factor: 350.886 || RMSE_lat: 45.208 || RMSE_ax: 46.871 || RMSE_x: 32.063 || RMSE_y: 31.871 || Cost: 12.990 || Recall: 0.806 || Precision: 0.997 || BatchNr.: 14200 || Time Upd.: 381.4 ms 
 || Count Cost: 14.8125 || Localization Cost: 3.1000 || Bg Cost: 0.0274 || P_locs Cost: 18.9345saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 60.086 || 45.208 || 46.871 || 14200


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10631FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10631 ,preds: 8704
JoR: 1.775 || Eff_3d: 60.02

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10631FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10631 ,preds: 8419
JoR: 1.740 || Eff_3d: 59.177 || Jaccard: 0.790 || Factor: 337.168 || RMSE_lat: 45.416 || RMSE_ax: 47.274 || RMSE_x: 32.722 || RMSE_y: 31.494 || Cost: 12.305 || Recall: 0.791 || Precision: 0.999 || BatchNr.: 15400 || Time Upd.: 388.4 ms 
 || Count Cost: 14.9682 || Localization Cost: 2.4911 || Bg Cost: 0.0258 || P_locs Cost: 19.0147saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 59.177 || 45.416 || 47.274 || 15400


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10631FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10631 ,preds: 8593
JoR: 1.772 || Eff_3d: 59.84

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10631FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10631 ,preds: 8615
JoR: 1.791 || Eff_3d: 59.764 || Jaccard: 0.806 || Factor: 355.868 || RMSE_lat: 45.025 || RMSE_ax: 49.612 || RMSE_x: 32.575 || RMSE_y: 31.082 || Cost: 13.542 || Recall: 0.808 || Precision: 0.997 || BatchNr.: 16600 || Time Upd.: 419.7 ms 
 || Count Cost: 14.7353 || Localization Cost: 3.5340 || Bg Cost: 0.0296 || P_locs Cost: 18.7834saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 59.764 || 45.025 || 49.612 || 16600


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10631FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10631 ,preds: 8868
JoR: 1.776 || Eff_3d: 60.30

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10631FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10631 ,preds: 8624
JoR: 1.776 || Eff_3d: 60.120 || Jaccard: 0.808 || Factor: 350.833 || RMSE_lat: 45.484 || RMSE_ax: 47.065 || RMSE_x: 32.886 || RMSE_y: 31.421 || Cost: 12.442 || Recall: 0.809 || Precision: 0.998 || BatchNr.: 17800 || Time Upd.: 393.5 ms 
 || Count Cost: 14.6580 || Localization Cost: 2.7933 || Bg Cost: 0.0254 || P_locs Cost: 18.7370saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 60.120 || 45.484 || 47.065 || 17800


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10631FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10631 ,preds: 8530
JoR: 1.775 || Eff_3d: 59.76

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10631FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10631 ,preds: 8479
JoR: 1.834 || Eff_3d: 60.870 || Jaccard: 0.796 || Factor: 338.890 || RMSE_lat: 43.415 || RMSE_ax: 44.897 || RMSE_x: 31.401 || RMSE_y: 29.980 || Cost: 12.371 || Recall: 0.797 || Precision: 0.999 || BatchNr.: 19000 || Time Upd.: 411.0 ms 
 || Count Cost: 14.7584 || Localization Cost: 2.6649 || Bg Cost: 0.0257 || P_locs Cost: 18.7948saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 60.870 || 43.415 || 44.897 || 19000


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 10631FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10631 ,preds: 8509
JoR: 1.835 || Eff_3d: 60.55

In [11]:
from cellbgnet.utils.plot_funcs import plot_train_record

In [12]:
plot_train_record(model)